In [1]:
import os
import numpy as np
import pandas as pd
import re

In [2]:
# set home directory
home_path = os.getcwd()[:os.getcwd().find('Code/python/obf_autax/bz')-1]
# set bz_path
bz_path = os.path.join(home_path, 'Data', 'obf_autax', 'nutz', 'nutz_171_12_ps_200907.xlsx')

print(home_path)
print(bz_path)

/home/philipp
/home/philipp/Data/obf_autax/nutz/nutz_171_12_ps_200907.xlsx


In [3]:
bz_data = pd.read_excel(bz_path)

In [4]:
bz_data.head()

,Waldort,WE-Typ,age,area,MA,FL,LH_ha,NH_ha,sum_ha,LH,NH,sum,DR,BH,ZP,SG,RU,Text
0,201A0,WO,141,2.794631,DF,2.5,15.000000,65.000000,80.000000,40,160,200,2.0,1.0,3.0,4.0,35.0,"Mitte Dezennium, bei Bundesstraße"
1,201B1,WO,216,2.707117,NaN,2.4,0.000000,0.000000,0.000000,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,201B2,WO,96,0.735919,DF,0.7,15.670690,65.000000,80.670690,10,50,60,2.0,1.0,3.0,4.0,35.0,"Mitte Dezennium, bei Bundesstraße"
3,201B3,WO,76,1.945348,DF,1.8,7.013899,63.125088,70.138987,10,110,120,2.0,1.0,3.0,4.0,35.0,"Mitte Dezennium, bei Bundesstraße"
4,201B4,WO,36,0.586157,DE,0.5,0.000000,55.000000,55.000000,0,30,30,2.0,1.0,3.0,4.0,35.0,"Mitte Dezennium, bei Bundesstraße"


In [5]:
# filter just WO
bz_data = bz_data.loc[bz_data['WE-Typ']=='WO', :]

In [6]:
bz_data.columns

Index(['Waldort', 'WE-Typ', 'age', 'area', 'MA', 'FL', 'LH_ha', 'NH_ha',
       'sum_ha', 'LH', 'NH', 'sum', 'DR', 'BH', 'ZP', 'SG', 'RU', 'Text'],
      dtype='object')

In [7]:
bz_data = bz_data[['Waldort', 'MA', 'FL', 'LH', 'NH', 'DR', 'BH', 'ZP', 'SG', 'RU', 'Text']]

In [8]:
bz_data.iloc[3,:]

Waldort                                201B3
MA                                        DF
FL                                       1.8
LH                                        10
NH                                       110
DR                                         2
BH                                         1
ZP                                         3
SG                                         4
RU                                        35
Text       Mitte Dezennium, bei Bundesstraße
Name: 3, dtype: object

In [9]:
# check for dublicates
try:
    pd.concat(g for _, g in bz_data.groupby("Waldort") if len(g) > 1)
except:
    print("no dublicates")

no dublicates


In [10]:
df = bz_data.copy()

In [11]:
# set raw_path
raw_path = os.path.join(home_path, 'Data', 'obf_autax', 'TO_1310_171_12_20200905_all.XLS')
# read tsv
wo_data = pd.read_csv(raw_path, sep='\t', encoding = "ISO-8859-1", decimal=',', error_bad_lines=False)

/home/philipp/miniconda3/envs/autax_rec/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (149) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
wos = wo_data['Waldort'].unique()
wos

array(['207A3', '207A2', '207A4', ..., '29631', '29641', '29632'],
      dtype=object)

In [13]:
len(wo_data[(wo_data['Waldort'] == '207A4') & (wo_data['Best.-Schicht.1'] != 0)])

3

In [14]:
wo_data.loc[wo_data['Best.-Schicht'] == 0, 'Bestockungsziel']

0        1FI 6BU 2AH 1TA
6        6BU 1TA 1FI 2AH
9        1FI 6BU 1TA 2AH
14               5BU 5SK
20               7SK 3BU
              ...       
10953                NaN
10954                NaN
10955                NaN
10956                NaN
10957                NaN
Name: Bestockungsziel, Length: 2111, dtype: object

In [15]:
wo_data.loc[wo_data['Best.-Schicht'] == 0, 'Bestockungsziel'].str.count(' ') + 1

0        4.0
6        4.0
9        4.0
14       2.0
20       2.0
        ... 
10953    NaN
10954    NaN
10955    NaN
10956    NaN
10957    NaN
Name: Bestockungsziel, Length: 2111, dtype: float64

In [16]:
# add bz_rows
wo_data['bz_rows'] = wo_data.loc[:,'Bestockungsziel'].str.count(' ') + 1

In [17]:
wo_data.loc[(wo_data['Waldort'] == '207A4') & (wo_data['Best.-Schicht'] == 0), 'bz_rows'].values[0]

4.0

## add number of BZ

In [18]:
wo_data.loc[(wo_data['Best.-Schicht'] == 0),['Waldort', 'bz_rows']]

,Waldort,bz_rows
0,207A3,4.0
6,207A2,4.0
9,207A4,4.0
14,204H0,2.0
20,203C0,2.0
...,...,...
10953,29541,NaN
10954,29651,NaN
10955,29631,NaN
10956,29641,NaN


In [19]:
bz_rows = wo_data.loc[(wo_data['Best.-Schicht'] == 0),['Waldort','bz_rows']]

In [20]:
df = df.merge(bz_rows, on='Waldort')

In [21]:
df

,Waldort,MA,FL,LH,NH,DR,BH,ZP,SG,RU,Text,bz_rows
0,201A0,DF,2.5,40,160,2.0,1.0,3.0,4.0,35.0,"Mitte Dezennium, bei Bundesstraße",2.0
1,201B1,NaN,2.4,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2,201B2,DF,0.7,10,50,2.0,1.0,3.0,4.0,35.0,"Mitte Dezennium, bei Bundesstraße",2.0
3,201B3,DF,1.8,10,110,2.0,1.0,3.0,4.0,35.0,"Mitte Dezennium, bei Bundesstraße",4.0
4,201B4,DE,0.5,0,30,2.0,1.0,3.0,4.0,35.0,"Mitte Dezennium, bei Bundesstraße",2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
815,307O2,NaN,1.1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,3.0
816,307O3,DE,0.7,0,30,2.0,1.0,3.0,6.0,36.0,"Mitte Dezennium, LA fördern, zusammen mit O3",3.0
817,307O4,NaN,0.5,0,0,NaN,NaN,NaN,NaN,NaN,NaN,3.0
818,307P1,NaN,3.2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,3.0


## add number of rows

In [22]:
wos = wo_data['Waldort'].unique()
df.set_index(['Waldort'], drop=False, inplace=True)

# add number of rows
for wo in wos:
    df.loc[wo, 'nr_rows'] = len(wo_data[(wo_data['Waldort'] == wo) & (wo_data['Best.-Schicht.1'] != 0)])

In [24]:
df.reset_index(drop=True, inplace=True)
df = df[['Waldort', 'bz_rows', 'nr_rows', 'MA', 'FL', 'LH', 'NH', 'DR', 'BH', 'ZP', 'SG', 'RU', 'Text']]

In [25]:
df

,Waldort,bz_rows,nr_rows,MA,FL,LH,NH,DR,BH,ZP,SG,RU,Text
0,201A0,2.0,10.0,DF,2.5,40.0,160.0,2.0,1.0,3.0,4.0,35.0,"Mitte Dezennium, bei Bundesstraße"
1,201B1,2.0,7.0,NaN,2.4,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,201B2,2.0,3.0,DF,0.7,10.0,50.0,2.0,1.0,3.0,4.0,35.0,"Mitte Dezennium, bei Bundesstraße"
3,201B3,4.0,5.0,DF,1.8,10.0,110.0,2.0,1.0,3.0,4.0,35.0,"Mitte Dezennium, bei Bundesstraße"
4,201B4,2.0,2.0,DE,0.5,0.0,30.0,2.0,1.0,3.0,4.0,35.0,"Mitte Dezennium, bei Bundesstraße"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2094,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2095,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2096,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2097,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## add structure from raw data export

In [26]:
# filter relevant data
wo_data = wo_data[wo_data['Best.-Schicht'] == 0]

### pernitz (comment if not aplicable)
wo_data = wo_data[wo_data['Abteilung'] > 200]

# sort data
wo_data.sort_values(by=['Abteilung', 'WE-Typ', 'Unterabteil.', 'Teilfl.'], ascending=[True,False,True,True], inplace=True)

# check if NG are included
print(wo_data['WE-Typ'].unique())

['WO' 'NG']


In [27]:
# merge data -> add NG
merge_data = wo_data[['Waldort', 'WE-Typ']].merge(df, how='left', on='Waldort')

In [28]:
merge_data

,Waldort,WE-Typ,bz_rows,nr_rows,MA,FL,LH,NH,DR,BH,ZP,SG,RU,Text
0,201A0,WO,2.0,10.0,DF,2.5,40.0,160.0,2.0,1.0,3.0,4.0,35.0,"Mitte Dezennium, bei Bundesstraße"
1,201B1,WO,2.0,7.0,NaN,2.4,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,201B2,WO,2.0,3.0,DF,0.7,10.0,50.0,2.0,1.0,3.0,4.0,35.0,"Mitte Dezennium, bei Bundesstraße"
3,201B3,WO,4.0,5.0,DF,1.8,10.0,110.0,2.0,1.0,3.0,4.0,35.0,"Mitte Dezennium, bei Bundesstraße"
4,201B4,WO,2.0,2.0,DE,0.5,0.0,30.0,2.0,1.0,3.0,4.0,35.0,"Mitte Dezennium, bei Bundesstraße"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,307O3,WO,3.0,3.0,DE,0.7,0.0,30.0,2.0,1.0,3.0,6.0,36.0,"Mitte Dezennium, LA fördern, zusammen mit O3"
1020,307O4,WO,3.0,2.0,NaN,0.5,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1021,307P1,WO,3.0,4.0,NaN,3.2,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1022,307P2,WO,4.0,7.0,NaN,1.5,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# format data
merge_data.fillna(0, inplace = True)
merge_data.iloc[:,2:4] = merge_data.iloc[:,2:4].astype(int)
merge_data.iloc[:,6:13] = merge_data.iloc[:,6:13].astype(int)

In [30]:
# set bz_path
save_path = os.path.join(home_path, 'Data', 'obf_autax', 'csv_for_autax', 'nutz_autax_1310_171_12_20200907.csv')
merge_data.to_csv(save_path, index=False)